In [1]:
import re
from datetime import date
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import numpy as np
import pandas as pd
import seaborn as sns

Things to keep in mind for the data:
-henddf and stadf don't include VIN
-henddf doesn't have price for a large portion of the data

In [2]:
henddf = pd.read_csv("selenium_hend_paginated_scrape.csv")
keffdf = pd.read_csv("selenium_kef_paginated_scrape.csv")
lkndf = pd.read_csv("selenium_LKN_paginated_scrape.csv")
statdf = pd.read_csv("selenium_stateline_paginated_scrape.csv")
nctoyodf =pd.read_csv("selenium_NcharToyo_paginated_scrape.csv")
modnisdf = pd.read_csv("selenium_mod_Nissan_LKN_paginated_scrape.csv")
lkninfdf = pd.read_csv("selenium_LKNinf_paginated_scrape.csv")
lknhyudf = pd.read_csv("selenium_LKNhyun_paginated_scrape.csv")

In [3]:
print(henddf.count())
print(keffdf.count())
print(lkndf.count())
print(statdf.count())

Condition_Year_Make_Model    1233
Price                         207
dtype: int64
Condition_Year    555
Make_Model        555
Price             545
Vin               555
dtype: int64
Condition_Year    1476
Make_Model        1476
Price             1435
Vin               1476
dtype: int64
Condition_Year    2331
Make_Model        2331
Price             2283
Vin                  0
dtype: int64


In [4]:
#formatted on site as Condition/Year/Make/Model/Trim
#source 1
henddf["Dealer"] = "1"


In [5]:
#formatted on site as Condition/Year then Make/Model/Trim
#source 2
keffdf["Dealer"] = "2"
lkndf["Dealer"] = "3"
modnisdf["Dealer"] = "6"

In [6]:
#formatted on site as "condition/year" then Condition/Year/Make/Model/Trim
#source 3
statdf["Dealer"] = "4"


In [7]:
#formatted on site as "condition" then Year/Make/Model/Trim
#source 4
nctoyodf["Dealer"] = "5"

In [8]:
#formatted on site as "condition" then "Year" then "Make/Model" then "Trim"
#source 5
lkninfdf["Dealer"] = "7"

In [9]:
#formatted on site as "condition" then "Year" then Make/Model/Trim
#source 6
lknhyudf["Dealer"] = "8"

In [10]:
def combine_source1(df):
    df["Condition_Year_Make_Model"] ="new" + " " + df["Condition_Year_Make_Model"]
    return df

In [11]:
def combine_source2(df):
    df["Condition_Year_Make_Model"]=df["Condition_Year"].fillna("") + " " +df["Make_Model"].fillna("")
    df=df.drop(["Condition_Year", "Make_Model"], axis=1)
    return df

In [12]:
def combine_source3(df):
    df["Condition_Year_Make_Model"]=df["Make_Model"]
    df=df.drop(["Condition_Year", "Make_Model"], axis=1)
    return df

In [13]:
def combine_source4(df):
    df["Condition_Year_Make_Model"]=df["Condition"] + " " + df["Year_Make_Model"]
    df=df.drop(["Condition", "Year_Make_Model"], axis=1)
    return df

In [14]:
def combine_source5(df):
    df["Condition_Year_Make_Model"]=df["Condition"] + " " + df["Year"].astype(str) + " " + df["Make_Model"]+ " " + df["Trim"]
    df=df.drop(["Condition", "Year", "Make_Model", "Trim"], axis=1)
    return df

In [15]:
def combine_source6(df):
    df["Condition_Year_Make_Model"]=df["Condition"] + " " + df["Year"].astype(str) + " " + df["Make_Model"]
    df=df.drop(["Condition", "Year", "Make_Model"], axis=1)
    return df

In [16]:
#function to normalize condition data and make everything lower case
def normalize_df_column(df, col):
    def normalize_text(text):
        text = text.lower()
        # Replace variations with "used"
        text = re.sub(r'\bcertified\s+pre[-\s]?owned\b', 'used', text)
        text = re.sub(r'\bpre[-\s]?owned\b', 'used', text)
        text = re.sub(r'\bcertified\s+used\b', 'used', text)
        # Remove extra spaces
        return re.sub(r'\s+', ' ', text).strip()

    df[col] = df[col].astype(str).apply(normalize_text)
    return df

In [17]:
#to run through what ever needs to be changed to make them uniform for prcessing later depending on the source
def  process_inventory_dataframe(df, source):
    if source=="source 1":
        df=combine_source1(df)
    elif source=="source 2":
        df=combine_source2(df)
    elif source=="source 3":
        df=combine_source3(df)
    elif source=="source 4":
        df=combine_source4(df)
    elif source=="source 5":
        df=combine_source5(df)
    elif source=="source 6":
        df=combine_source6(df)

    return df

In [18]:
#run pipeline function and combine dataframes
henddf_clean = process_inventory_dataframe(henddf, source="source 1")
keffdf_clean = process_inventory_dataframe(keffdf, source="source 2")
lkndf_clean = process_inventory_dataframe(lkndf, source="source 2")
statdf_clean = process_inventory_dataframe(statdf, source="source 3")
nctoyodf_clean = process_inventory_dataframe(nctoyodf, source="source 4")
modnisdf_clean = process_inventory_dataframe(modnisdf, source="source 3")
lkninfdf_clean = process_inventory_dataframe(lkninfdf, source="source 5")
lknhyudf_clean = process_inventory_dataframe(lknhyudf, source="source 6")

#combine the different dataframes and normalize the Condition/Year/make/model column
comb_df = pd.concat([henddf_clean, keffdf_clean, lkndf_clean, statdf_clean, nctoyodf_clean, modnisdf_clean, lkninfdf_clean, lknhyudf_clean], ignore_index=True)
comb_df = normalize_df_column(comb_df, "Condition_Year_Make_Model")
comb_df.to_csv("comb_dealer_data.csv", index=False)

In [19]:
comb_df.head()

,Condition_Year_Make_Model,Price,Dealer,Vin,Stock
0,new 2025 jeep grand cherokee laredo x,"$39,580",1,NaN,NaN
1,new 2025 dodge durango gt,"$40,090",1,NaN,NaN
2,new 2025 jeep grand cherokee laredo x,"$40,175",1,NaN,NaN
3,new 2025 jeep grand cherokee laredo x,NaN,1,NaN,NaN
4,new 2025 jeep grand cherokee laredo x,NaN,1,NaN,NaN


In [20]:
def pull_condition(df):
    # Split Condition/Year/make/modle at the first space space
    split_at_cond = comb_df["Condition_Year_Make_Model"].str.split(" ", n=1, expand=True).fillna("")
    df["Condition"] = split_at_cond[0]
    df["Year_Make_Model"] = split_at_cond[1]

    #check for any condition words in the leftover column and delete them
    pattern = re.compile(r'\b(new|used|certified|gold|pre[-\s]?owned)\b', flags=re.IGNORECASE)
    df["Year_Make_Model"] = df["Year_Make_Model"].astype(str).apply(lambda x: pattern.sub('', x).strip())
    #drop old column
    df=df.drop(["Condition_Year_Make_Model"], axis=1)
    #Capitalize first letter
    df["Condition"] = df["Condition"].astype(str).str.title()
    return df

In [21]:
def pull_year(df):
    #split 
    split_at_cond = comb_df["Year_Make_Model"].str.split(" ", n=1, expand=True).fillna("")
    comb_df["Year"] = split_at_cond[0]
    comb_df["Make_Model"] = split_at_cond[1]
    df=df.drop(["Year_Make_Model"], axis=1)
    return df

In [22]:
def pull_make(df):
    known_makes = {'jeep', 'ram', 'chrysler', 'dodge', 'fiat', 'ford', 'hyundai', 'honda', 'kia',
 'toyota', 'nissan', 'audi', 'lexus', 'chevrolet', 'bmw', 'gmc', 'tesla', 'volvo',
 'subaru', 'genesis', 'infiniti', 'alfa romeo', 'land rover', 'scion', 'mazda', 'lincoln', 'acura', 'mercedes-benz', 'volkswagen', 'porsche', 'mini',
'mitsubishi', 'buick', 'coleman', 'crossroads', 'cadillac', 'freightliner', 'maserati', 'jaguar', 'bentley', 'ineos'}
    def checker(text):
        text = str(text).strip().lower()

        # Try matching entire known make at start
        for make in known_makes:
            if text.startswith(make):
                model = text[len(make):].strip()
                return pd.Series([make, model])

        # Fallback: split by first space
        parts = text.split()
        if len(parts) >= 2:
            return pd.Series([parts[0], " ".join(parts[1:])])
        elif len(parts) == 1:
            return pd.Series([parts[0], ""])  # one word, assume it's the make
        else:
            return pd.Series(["unknown_make", "unknown_model"])  # empty/malformed row

    #run checker through column
    df[["Make", "Model_Trim"]] = df["Make_Model"].apply(checker)
    #drop old column
    df=df.drop(["Make_Model"], axis=1)
    #Capitalize first letter
    df["Make"] = df["Make"].astype(str).str.title()
    return df

In [23]:
def split_model_trim(df):
    use_two_word_models = ["grand", "model", "flying", "range", "land", "promaster"]
    use_three_word_models = ["town"]
    drop_first_word_models = ["trucks"]


    def custom_model_trim_split(text):
        words = text.split()
        if not words:
            return ("", "")

        # Drop the first word if it's in the drop list
        if words[0] in drop_first_word_models:
            words = words[1:]

        # Check 3-word model rules
        if words and words[0] in use_three_word_models and len(words) >= 3:
            model = " ".join(words[:3])
            trim = " ".join(words[3:]) if len(words) > 3 else ""
            return (model, trim)

        # Check 2-word model rules
        if words and words[0] in use_two_word_models and len(words) >= 2:
            model = " ".join(words[:2])
            trim = " ".join(words[2:]) if len(words) > 2 else ""
            return (model, trim)

        # Default: first word is model, rest is trim
        model = words[0]
        trim = " ".join(words[1:]) if len(words) > 1 else ""
        return (model, trim)


    # Apply the function
    comb_df[["Model", "Trim"]] = comb_df["Model_Trim"].apply(
        lambda x: pd.Series(custom_model_trim_split(x))
    )
    # Drop the original combined column
    comb_df.drop(columns="Model_Trim", inplace=True)
        #Capitalize first letter
    df["Model"] = df["Model"].astype(str).str.title()
    
    return df


In [24]:
comb_df=pull_condition(comb_df)

In [25]:
comb_df=pull_year(comb_df)

In [26]:
comb_df=pull_make(comb_df)

In [27]:
comb_df=split_model_trim(comb_df)

In [28]:
def format_make(make):
    # List of known Make acronyms that should stay fully uppercase
    acronyms = [
        "BMW",
        "GMC",
        "RAM",
        "KIA",
        "FIAT",
        "MINI",
        "BYD",
        "MG",
        "HUMMER",
        "GENESIS",
    ]
    make_upper = make.upper()
    if make_upper in acronyms:
        return make_upper
    else:
        return make.capitalize()


# Apply the function
comb_df.loc[:, "Make"] = comb_df["Make"].apply(format_make)

In [29]:
def format_make(model):
    acronyms = [
        "CR-V",
        "ES",
        "IS",
        "LS",
        "NX",
        "RX",
        "GLC",
        "C-HR",
        "MDX",
        "ILX",
        "ZDX",
        "HR-V",
        "MKZ",
        "SRX",
        "XT6",
        "XT5",
        "XT4",
        "F-150",
        "F-250SD",
        "F-350SD",
        "GLE",
        "ML",
        "TL",
        "TLX",
        "RLX",
        "CT",
        "EQB",
        "EQS",
        "G90",
        "G80",
        "GV70",
        "GV80",
        "G70",
        "Q3",
        "Q5",
        "Q7",
        "Q8",
        "QX50",
        "QX55",
        "QX60",
        "QX80",
        "XC40",
        "XC60",
        "XC90",
        "A3",
        "A4",
        "A5",
        "A6",
        "A7",
        "A8",
        "S4",
        "S5",
        "S60",
        "SQ5",
        "CLE",
        "GLA",
        "GLB",
        "GLS",
        "AMG®",
        "C",
        "E",
        "G",
        "CLA",
        "GR",
        "Z4",
        "Z",
        "X1",
        "X3",
        "X5",
        "X7",
        "i",
        "i4",
        "iX",
        "3",
        "5",
        "7",
        "8",
        "4",
        "EQB",
        "CX-30",
        "CX-5",
        "CX-50",
        "CX-70",
        "CX-9",
        "CX-90",
        "C-Max",
        "3.5L",
        "350Z",
    ]
    model_upper = model.upper()
    if model_upper in acronyms:
        return model_upper
    else:
        return model.capitalize()


# Apply the function
comb_df.loc[:, "Model"] = comb_df["Model"].apply(format_make)

In [30]:
comb_df["Date"]=date.today()
comb_df.to_csv("LocalInv.csv", index=False)

In [31]:
# boilerplate to hunt down iregular values
# list of problematic values
# Problem_children = ["Land"]  # replace with your actual values

# Filter rows where column is in the list
# problem_rows = comb_df[comb_df["Column"].isin(Problem_children)]

# Display the result
# print(problem_rows)